In [2]:
from evaluator_fast import Evaluate
from consistency_checker_fast import SemanticConsistnecyCheck
import time
from tqdm import tqdm
# from sklearn.metrics import roc_auc_score
import openai
import json
import openai
# from sklearn.metrics import precision_recall_curve, auc, classification_report, roc_auc_score
from tqdm import tqdm

# api_key = 
# openai.api_key = 'sk-S6lJtVwrvPOyWV9L11DcFb729bC6488e95493d368bF5B4A5'
# qa_data = load_dataset("json", data_files="dataset/hotpotQA_halu.json")

def hallucination_score(question, target_answer, model, num_samples):
    
    # llm evaluation
    llm_evaluate = Evaluate(model=model)
    scc = SemanticConsistnecyCheck(model=model)
    # fast self-evaluation
    fast_self_responses = llm_evaluate.self_evaluate_api(self_question = question, temperature = 1.0, self_num = num_samples)
    # fast consistency checker 
    fast_consistency_res = scc.score_scc_api(question, target_answer, candidate_answers = fast_self_responses, temperature = 0.0)
    
    return fast_consistency_res[0]

/home/wxy/.conda/envs/sac3/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [14]:


halu_score_all = []
true_label = []

import os
# start = 0
# end = 20
# data_name = 'hotpot_qa'
data_name = 'gsm8k'
# model_name = 'vicuna-7b'
# model_name = 'llama2-7b-chat-hf'
# model_name = 'Mistral-7B-Instruct-v0.2'
model_name = 'vicuna-13b'

test_path = f'/home/wxy/project/hallucination_predict_by_prompt/data/datasets/gsm8k/0shot/output/0shot/gsm8k_{model_name}_1_sample_0shot_test.json'
output_file = f'./result/result_{data_name}_{model_name}_v2.txt'
output_time_file = f'./result/time_cost_{data_name}_{model_name}_v2.txt'

In [15]:
model = 'gpt-3.5-turbo'
num_samples = 10
d_time = {}
with open(test_path, 'r') as f, open(output_file, 'a') as w, open(output_time_file, 'a') as t:
    for line in f.readlines():
        d = json.loads(line.strip())
        q_id = d['id']
        target_answer =  d['predict']
        label = d['label']
        question = d['prompt'] 
        t0 = time.time()
        halu_score = hallucination_score(question, target_answer, model, num_samples)
        time_cost = time.time() - t0
        w.write(f'{q_id}\t{halu_score}\t{label}\n')
        print(f'{q_id}\t{halu_score}\t{label}\t{time_cost:.4f}')
        t.write(f'{q_id}\t{time_cost:.4f}\n')

        time.sleep(1)

800	0.8	1	5.6967
801	0.5	0	5.0160
802	0.8	1	6.4302
803	0.0	0	4.4819
804	0.1	1	4.1672
805	0.3	1	6.4814
806	0.8	1	4.3321
807	0.0	0	4.5339
808	0.6	1	3.9644
809	0.0	0	4.2480
810	0.1	1	5.6174
811	0.5	1	6.0279
812	0.0	0	3.7941
813	0.2	1	4.0396
814	0.8	1	6.3681
815	0.4	1	5.6627
816	0.0	1	3.7510
817	0.8	1	4.7182
818	0.6	1	4.9839
819	0.4	1	4.7687
820	0.2	1	5.9346
821	0.9	1	4.1160
822	0.7	1	4.7388
823	0.7	1	4.7752
824	1.0	1	5.7781
825	0.6	1	3.7763
826	0.0	0	5.0949
827	0.1	1	5.6508
828	0.0	0	4.1867
829	0.5	1	5.2901
830	0.2	1	5.2412
831	0.9	1	4.6728
832	0.4	1	4.3951
833	0.3	1	4.7476
834	0.0	0	5.1916
835	0.8	1	5.5953
836	1.0	1	3.3782
837	0.1	0	3.0947
838	0.0	0	6.2313
839	1.0	1	4.0311
840	0.6	1	5.8620
841	0.0	0	4.4196
842	0.0	0	3.4076
843	0.8	1	4.8106
844	0.0	0	5.5187
845	0.5	1	4.2601
846	0.7	1	8.0139
847	0.1	0	4.0481
848	0.0	0	6.4670
849	0.8	1	3.8253
850	0.1	1	7.1441
851	0.8	1	7.2543
852	0.1	0	3.6176
853	0.6	1	3.5376
854	0.4	1	7.9208
855	0.5	1	5.2348
856	1.0	1	3.9656
857	0.0	0	6.4758
858	0.0	0	4.21

In [ ]:
d_time